# Coder Agent Workspace

Drive the lightweight `CoderAgent` to spin up focused Python projects with your local tool-calling LLM.

## Imports & configuration

Set the endpoint, model, and folders once, then reuse the agent across briefs.

In [1]:
from core.agent import CoderAgent
from core.config import LLMConfig
from pathlib import Path
import json
BASE_URL = "http://localhost:1234/v1"
MODEL = "qwen/qwen3-coder-30b"
PROJECT_ROOT = "generated_projects"
LOG_DIR = "logs"
print(f"Base URL: {BASE_URL}")
print(f"Model: {MODEL}")
print(f"Project root: {PROJECT_ROOT}")
print(f"Log dir: {LOG_DIR}")


Base URL: http://localhost:1234/v1
Model: qwen/qwen3-coder-30b
Project root: generated_projects
Log dir: logs


## Instantiate the coder agent

This keeps the workspace isolated and exposes filesystem tools to the LLM.

In [2]:
config = LLMConfig.load()
config.base_url = BASE_URL
config.model = MODEL
config.log_dir = LOG_DIR

coder = CoderAgent(
    config=config,
    project_root=PROJECT_ROOT,
    verbose=False,
)

print("Coder agent ready.")


Coder agent ready.


## Set the project brief

Use this cell to describe the project you want the coder agent to build. It currently defaults to a React calculator demo so you can validate the workflow quickly.


In [3]:
# project_brief = """Build a simple React calculator web app.

# Requirements:
# - Use React 18 via CDN (no bundler) with a single HTML host page.
# - Implement a calculator UI with digits, basic operators, equals, AC, and delete actions.
# - Style it so it looks modern on desktop and mobile.
# - Include a README with steps to run a simple static server.
# """
# project_name = "react_calculator"  # Set to None to auto-generate from the brief
# max_iterations = 6

# print("Current project name:", project_name)
# print(project_brief)


## Build the project

The agent loops through chat + tool calls until it finishes or hits the iteration cap.

## Build: Personal modern web app (with skills)

This triggers the agent to build a personal modern web app using the `personal_website` skill plus context.


In [4]:
# Build a personal modern web app using selected skills
personal_web_brief = """Build a personal modern web app with react js."""
skills = ["personal_website"]
coder.reset_conversation()
personal_summary = coder.build_project(
    project_description=personal_web_brief,
    project_name="personal_web",
    max_iterations=50,
    skills=skills,
)
print("Build summary:", personal_summary)


Using base_instruction: You are a focused software engineer.
            Build a small but complete project inside '/Users/Khaled.Alabsi/projects/llm-agent/generated_projects/personal-web'.

            - Follow the brief precisely and avoid hidden fallbacks.
            - Use the available tools to plan, create files, and run shell commands when needed.
            - Prefer minimal dependencies and a clear, runnable structure.
            - Provide a concise summary at the end with run instructions.

            Reference material and skills:
            ## agent_notes.md




## no_svg.md

# No-Go: SVG Generation

Never generate or embed SVG assets. When graphics are needed, use a placeholder image URL or insert a clear `TODO: add image` note instead. This rule applies to inline `<svg>` elements and standalone `.svg` files.



## coder.md

# Skill: General Coder Orchestrator

Purpose: Provide stack-agnostic guidance for assembling small but complete software projects from a natural-la